In [26]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.compat.v1.disable_eager_execution()
import matplotlib.pyplot as plt
import numpy as np
import random

In [28]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot = True)

X = tf.placeholder(tf.float32,[None,784])
X_img = tf.reshape(X,[-1,28,28,1])
Y = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)
W1 = tf.Variable(tf.random_normal([3,3,1,32],stddev = 0.01))
L1 = tf.nn.conv2d(X_img,W1,strides=[1,1,1,1],padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1,ksize=[1,2,2,1],strides = [1,2,2,1],padding = 'SAME')
L1 = tf.nn.dropout(L1,keep_prob=keep_prob)
print(L1.shape)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(?, 14, 14, 32)


In [29]:
W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev = 0.01))
L2 = tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
L2 = tf.nn.dropout(L2,keep_prob=keep_prob)
print(L2.shape)

(?, 7, 7, 64)


In [30]:
W3 = tf.Variable(tf.random_normal([3,3,64,128],stddev = 0.01))
L3 = tf.nn.conv2d(L2,W3,strides = [1,1,1,1],padding='SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3,ksize=[1,2,2,1],strides = [1,2,2,1],padding='SAME')
L3 = tf.nn.dropout(L3,keep_prob=keep_prob)
print(L3.shape)

(?, 4, 4, 128)


In [31]:
W4 = tf.Variable(tf.random_normal([3,3,128,256],stddev = 0.01))
L4 = tf.nn.conv2d(L3,W4,strides = [1,1,1,1],padding='SAME')
L4 = tf.nn.relu(L4)
L4 = tf.nn.max_pool(L4,ksize=[1,2,2,1],strides = [1,2,2,1],padding='SAME')
print(L4.shape)
L4 = tf.reshape(L4,[-1,2*2*256])

(?, 2, 2, 256)


In [32]:
W5 = tf.Variable(tf.random_normal([2*2*256,10],stddev = 0.01))
b = tf.Variable(tf.random_normal([10]))

In [33]:
hypothesis = tf.matmul(L4,W5) +b
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = hypothesis, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)

In [35]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('learning started. It may take some time')
for epoch in range(15):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / 100)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        feed_dict = {X:batch_xs,Y:batch_ys,keep_prob:0.7}
        c,_ = sess.run([cost,optimizer],feed_dict = feed_dict)
        avg_cost += c/total_batch
    print('Epoch:','%04d'%(epoch+1),'cost','{:.9f}'.format(avg_cost))

print('learning finished')

correct_prediction = tf.equal(tf.argmax(hypothesis,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print('accuracy:',sess.run(accuracy,feed_dict = {X:mnist.test.images,Y:mnist.test.labels,keep_prob:1}))

learning started. It may take some time
Epoch: 0001 cost 0.634356817
Epoch: 0002 cost 0.139525411
Epoch: 0003 cost 0.089043317
Epoch: 0004 cost 0.065739672
Epoch: 0005 cost 0.052795109
Epoch: 0006 cost 0.047450264
Epoch: 0007 cost 0.041224029
Epoch: 0008 cost 0.036669804
Epoch: 0009 cost 0.034245898
Epoch: 0010 cost 0.032587701
Epoch: 0011 cost 0.030094240
Epoch: 0012 cost 0.027348332
Epoch: 0013 cost 0.027357421
Epoch: 0014 cost 0.023744077
Epoch: 0015 cost 0.023124619
learning finished
accuracy: 0.9931
